# Ratios code generator

In [1]:
import sys,os
import subprocess
import numpy as np
from numpy import abs
import pylab as py
import matplotlib.cm as cm
import sympy as sp
from sympy.printing.mathml import print_mathml,mathml
from sympy.parsing.sympy_parser import parse_expr
from sympy.tensor.array import MutableDenseNDimArray as Array
import copy
from sympy.utilities.lambdify import lambdastr

### utilities

In [2]:
from IPython.display import display, Math, Latex
import sympy as sp

def lpprint(expression): 
    return display(Math(sp.latex(expression)))


### vector class to handle 4-vectors

In [3]:
class Vec: 

    def __init__(self,symbol,rep='LC',mod=[1,1,1,1],theta=None):
        if theta==None: theta=sp.symbols(r'\theta_{%s}'%symbol)
        if rep=='LC': self.vec=self.LC(symbol,mod=mod,theta=theta)
        if rep=='MC': self.vec=self.MC(symbol,mod=mod,theta=theta)
        self.rep=rep

    def __add__(self,other):
        dum=Vec('dum',self.rep)
        dum.vec=[self.vec[i]+other.vec[i] for i in range(len(other.vec))]
        return dum

    def __sub__(self,other):
        #print self.vec
        #print other.vec
        dum=Vec('dum',self.rep)
        dum.vec=[self.vec[i]-other.vec[i] for i in range(len(other.vec))]
        return dum

    def __eq__(self,other):
        dum=Vec('dum',self.rep)
        dum.vec=[self.vec[i] for i in range(len(other.vec))]
        return dum
 
    def __mul__(self,other):
        if isinstance(other, Vec): 
            return self.dot(other.vec,self.vec)
        else:
            dum=Vec('dum',self.rep)
            dum.vec=[other*self.vec[i] for i in range(len(self.vec))]
            return dum
        
    def __rmul__(self,other):
        if isinstance(other, Vec): 
            return self.dot(other.vec,self.vec)
        else:
            dum=Vec('dum',self.rep)
            dum.vec=[other*self.vec[i] for i in range(len(self.vec))]
            return dum
        
    def LC(self,symbol,mod=[1,1,1,1],theta=None):
        p=sp.S('%s_p'%symbol)
        m=sp.S('%s_m'%symbol)
        t=sp.symbols('%s_t'%symbol,positive = True, real = True)
        if theta==None: theta=sp.S('theta_%s'%symbol)
        vec=[p,m,t*sp.cos(theta),t*sp.sin(theta)]
        self.p=p
        self.m=m
        self.t=t
        return [vec[i]*mod[i] for i in range(4)]
    
    def MC(self,symbol,mod=[1,1,1,1],theta=None):
        E=sp.symbols('%s_E'%symbol,positive = True, real = True)
        z=sp.symbols('%s_z'%symbol,real = True)
        t=sp.symbols('%s_t'%symbol,positive = True, real = True)
        if theta==None: theta=sp.S('theta_%s'%symbol)
        vec=[E,t*sp.cos(theta),t*sp.sin(theta),z]
        self.E=E
        self.z=z
        self.t=t
        self.theta=theta
        return [vec[i]*mod[i] for i in range(4)]

    def dot(self,A,B):
        if self.rep=='LC':
            return A[0]*B[1]+A[1]*B[0]-A[2]*B[2]-A[3]*B[3]
        elif self.rep=='MC':
            return A[0]*B[0]-A[1]*B[1]-A[2]*B[2]-A[3]*B[3]
        
    def set_plus(self,plus):
        self.vec[0]=plus
        
    def set_minus(self,minus):
        self.vec[1]=minus
        
    def set_perp(self,perp):
        self.vec[2:]=perp
        
    def set_E(self,E):
        self.vec[0]=E
        
    def set_x(self,x):
        self.vec[1]=x
        
    def set_y(self,y):
        self.vec[2]=y
        
    def set_z(self,z):
        self.vec[3]=z

    def get_E(self):
        return self.vec[0]
        
    def get_x(self):
        return self.vec[1]
        
    def get_y(self):
        return self.vec[2]
        
    def get_z(self):
        return self.vec[3]
    
    def change_plus(self,M):
        if  self.rep=='LC':
            perp2=(self.vec[2]**2+self.vec[3]**2)#.simplify()
            self.vec[0]=(perp2+M**2)/2/self.vec[1]
        
    def change_minus(self,M):
        if  self.rep=='LC':
            perp2=(self.vec[2]**2+self.vec[3]**2)#.simplify()
            self.vec[1]=(perp2+M**2)/2/self.vec[0]

    def LC2MC(self):
        if  self.rep=='LC':
            vec0=((self.vec[0]+self.vec[1])/sp.sqrt(2))#.simplify()
            vec3=((self.vec[0]-self.vec[1])/sp.sqrt(2))#.simplify()
            self.vec=[vec0,self.vec[2],self.vec[3],vec3]
            self.rep='MC'

    def MC2LC(self):
        if  self.rep=='MC':
            vec_p=((self.vec[0]+self.vec[3])/sp.sqrt(2))#.simplify()
            vec_m=((self.vec[0]-self.vec[3])/sp.sqrt(2))#.simplify()
            self.vec=[vec_p,vec_m,self.vec[1],self.vec[2]]
            self.rep='LC'

    def norm(self):
        return sp.sqrt(sum([self.vec[i]**2 for i in range(1,4)]))#.simplify()
        
    def uvec(self):
        dum=Vec('dum','MC')
        dum.set_E(sp.S(0))
        norm=self.norm()
        dum.vec[1:]=[self.vec[i]/norm for i in range(1,len(self.vec))]
        return dum
    
    def get_mass(self):
        if self.rep=='MC':
            return sp.sqrt(self.vec[0]**2-self.norm()**2)
        elif self.rep=='LC':
            return sp.sqrt(2*self.vec[0]*self.vec[1]-self.vec[2]**2-self.vec[3]**2)

    def get_sqmass(self):
        if self.rep=='MC':
            return self.vec[0]**2-self.norm()**2
        elif self.rep=='LC':
            return 2*self.vec[0]*self.vec[1]-self.vec[2]**2-self.vec[3]**2

    @staticmethod
    def dot3(A,B):
        return sum([A.vec[i]*B.vec[i] for i in range(1,4)])
    
    @staticmethod
    def cross(A,B):
        dum=Vec('dum','MC')
        dum.set_E(sp.S(0))
        x= A.vec[2]*B.vec[3]-A.vec[3]*B.vec[2]
        y=-A.vec[1]*B.vec[3]+A.vec[3]*B.vec[1]
        z= A.vec[1]*B.vec[2]-A.vec[2]*B.vec[1]
        dum.vec[1:]=[x,y,z]
        return dum
    
    def simplify_vec(self):
        for i in range(4):
            self.vec[i]=self.vec[i].simplify()

    def subs_vec(self,A,B):
        for i in range(4):
            self.vec[i]=self.vec[i].subs(A,B)
    
    def rotate(self,K,cos,sin):
        #--rootate around K 
        dum=Vec('dum','MC')
        UK=K.uvec()
        dum.vec[1:]=(self*cos+self.cross(UK,self)*sin+UK*(self.dot3(UK,self)*(1-cos))).vec[1:]
        dum.vec[0]=self.vec[0]
        return dum
        
    def get_cos(self,K):
        #--get cos along K
        UK=K.uvec()
        UV=self.uvec()
        return self.dot3(UV,UK)
    
    def get_sin(self,K):
        #--get sin along K
        UK=K.uvec()
        UV=self.uvec()
        return self.cross(UV,UK).norm()
        
    @staticmethod
    def get_boost_CS(A,bA):
        #--get cosh and sinsh for A -> boosted A (bA)
        #--A and bA are 2D vectors 0: energy, 1: momentum along boost dir
        cosh=(bA[1]*A[1]-bA[0]*A[0])/(A[1]**2-A[0]**2)
        if A[1]==0: sinh=(bA[1]-cosh*A[1])/A[0]
        else:       sinh=(bA[0]-cosh*A[0])/A[1]
        return cosh,sinh
        
    def boost(self,V,C=None,S=None):
        
        if C==None and S==None:
            #--boost to rest frame of V
            E=V.vec[0]
            P=V.norm()
            M=V.get_mass()
            C,S=self.get_boost_CS([E,P],[M,0])
            
        UV=V.uvec()
        par=self.dot3(self,UV)*UV
        per=self-par
        E=self.vec[0]
        npar=self.dot3(self,UV)#par.norm()
        #npar=par.norm()
        bE  =(C*E + S*npar)#.simplify()
        bpar=(S*E + C*npar)#.simplify()
        new=per+bpar*UV
        new.vec[0]=bE
        return new


In [4]:
A=Vec('A','LC')
sp.Matrix(A.vec)

Matrix([
[                A_p],
[                A_m],
[A_t*cos(\theta_{A})],
[A_t*sin(\theta_{A})]])

### built relevant sympy symbols

In [5]:
xb=sp.symbols('x_bj',positive = True, real = True)
xN=sp.symbols('x_N',positive = True, real = True)
zh=sp.symbols('z_h',positive = True, real = True)
zN=sp.symbols('z_N',positive = True, real = True)
xh=sp.symbols('x_h',positive = True, real = True)

Q=sp.symbols('Q',positive = True, real = True)
M=sp.symbols('M',positive = True, real = True)
Mh=sp.symbols('M_h',positive = True, real = True)
Mki=sp.symbols('M_ki',positive = True, real = True)

#Mki2=sp.symbols('M_ki2',positive = True, real = True)

Mkf=sp.symbols('M_kf',positive = True, real = True)

#Mkf2=sp.symbols('M_kf2',positive = True, real = True)


Mx=sp.symbols('M_x',positive = True, real = True)

MhT=sp.symbols('M_hT',positive = True, real = True)
qt=sp.S('q_t')
eps=sp.S('epsilon')

xi=sp.symbols('xi',positive = True, real = True)
zeta=sp.symbols('zeta',positive = True, real = True)
hxN=sp.symbols('\hat{x}_N',positive = True, real = True)
hzN=sp.symbols('\hat{z}_N',positive = True, real = True)


### construct relevant 4-vectors

In [6]:
ki=Vec('k_i','LC')
kf=Vec('k_f','LC')
kx=Vec('k_x','LC')

q=Vec('q','LC',mod=[1,1,0,0])
P=Vec('P','LC',mod=[1,1,0,0])
H=Vec('H','LC')
T=Vec('T',mod=[0,0,1,1],theta=sp.symbols(r'\theta_H'))
dk=Vec('delta_k',mod=[0,0,1,1])

q.set_plus(-Q/sp.sqrt(2))
q.set_minus(Q/sp.sqrt(2))
P.set_plus(Q/xN/sp.sqrt(2))
P.set_minus(xN*M**2/sp.sqrt(2)/Q)
H.set_minus(zN*q.vec[1])
H.set_perp((-zN*T).vec[2:])
H.change_plus(Mh)

ki.set_plus((xN/hxN)*P.vec[0])
kf.set_minus(H.vec[1]/(zN/hzN))
kf.set_perp((-hzN*T+dk).vec[2:])

ki.change_minus(Mki)
kf.change_plus(Mkf)


In [7]:
sp.Matrix(T.vec)

Matrix([
[                0],
[                0],
[T_t*cos(\theta_H)],
[T_t*sin(\theta_H)]])

In [8]:
sp.Matrix(H.vec)

Matrix([
[sqrt(2)*(M_h**2/2 + T_t**2*z_N**2*sin(\theta_H)**2/2 + T_t**2*z_N**2*cos(\theta_H)**2/2)/(Q*z_N)],
[                                                                                 sqrt(2)*Q*z_N/2],
[                                                                          -T_t*z_N*cos(\theta_H)],
[                                                                          -T_t*z_N*sin(\theta_H)]])

### build $x_N$ and $z_N$ 

In [9]:
zN2zh=sp.solve(zh-2*xb*(P*H)/Q**2,zN)
zN2zhA=zN2zh[1].simplify()
zN2zhB=zN2zh[0].simplify()

xN2xb=sp.solve(xb-Q**2/(2*P*q),xN)[0]
xN2xb=xN2xb*(Q+sp.sqrt(Q**2+4*M**2*xb**2))
xN2xb=xN2xb.expand()
xN2xb/=(Q+sp.sqrt(Q**2+4*M**2*xb**2))


In [10]:
zN2zhA

x_N*(Q**4*z_h + sqrt(-4*M**4*M_h**2*T_t**2*x_N**2*x_bj**2 - 4*M**2*M_h**2*Q**4*x_bj**2 + Q**8*z_h**2))/(2*x_bj*(M**2*T_t**2*x_N**2 + Q**4))

In [11]:
zN2zhB

x_N*(Q**4*z_h - sqrt(-4*M**4*M_h**2*T_t**2*x_N**2*x_bj**2 - 4*M**2*M_h**2*Q**4*x_bj**2 + Q**8*z_h**2))/(2*x_bj*(M**2*T_t**2*x_N**2 + Q**4))

In [12]:
xN2xb

2*Q*x_bj/(Q + sqrt(4*M**2*x_bj**2 + Q**2))

### build $k_X$ and $k$

In [13]:
kx=(ki+q-kf)
kx.simplify_vec()
sp.Matrix(kx.vec)

Matrix([
[sqrt(2)*(-Q**2*\hat{x}_N*\hat{z}_N + Q**2*\hat{z}_N - \hat{x}_N*(M_kf**2 + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2))/(2*Q*\hat{x}_N*\hat{z}_N)],
[                                                                                                                                 sqrt(2)*(Q**2*(1 - \hat{z}_N) + \hat{x}_N*(M_ki**2 + k_i_t**2))/(2*Q)],
[                                                                                                               T_t*\hat{z}_N*cos(\theta_H) - delta_k_t*cos(\theta_{delta_k}) + k_i_t*cos(\theta_{k_i})],
[                                                                                                               T_t*\hat{z}_N*sin(\theta_H) - delta_k_t*sin(\theta_{delta_k}) + k_i_t*sin(\theta_{k_i})]])

In [14]:
Mx2_val=kx.get_sqmass()
Mx2_val

-(T_t*\hat{z}_N*sin(\theta_H) - delta_k_t*sin(\theta_{delta_k}) + k_i_t*sin(\theta_{k_i}))**2 - (T_t*\hat{z}_N*cos(\theta_H) - delta_k_t*cos(\theta_{delta_k}) + k_i_t*cos(\theta_{k_i}))**2 + (Q**2*(1 - \hat{z}_N) + \hat{x}_N*(M_ki**2 + k_i_t**2))*(-Q**2*\hat{x}_N*\hat{z}_N + Q**2*\hat{z}_N - \hat{x}_N*(M_kf**2 + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2))/(Q**2*\hat{x}_N*\hat{z}_N)

In [15]:
k=kf-q
sp.Matrix(k.vec)

Matrix([
[sqrt(2)*Q/2 + sqrt(2)*(M_kf**2/2 + (-T_t*\hat{z}_N*sin(\theta_H) + delta_k_t*sin(\theta_{delta_k}))**2/2 + (-T_t*\hat{z}_N*cos(\theta_H) + delta_k_t*cos(\theta_{delta_k}))**2/2)/(Q*\hat{z}_N)],
[                                                                                                                                                            sqrt(2)*Q*\hat{z}_N/2 - sqrt(2)*Q/2],
[                                                                                                                                 -T_t*\hat{z}_N*cos(\theta_H) + delta_k_t*cos(\theta_{delta_k})],
[                                                                                                                                 -T_t*\hat{z}_N*sin(\theta_H) + delta_k_t*sin(\theta_{delta_k})]])

### Print kf, ki

In [16]:
sp.Matrix(kf.vec)

Matrix([
[sqrt(2)*(M_kf**2/2 + (-T_t*\hat{z}_N*sin(\theta_H) + delta_k_t*sin(\theta_{delta_k}))**2/2 + (-T_t*\hat{z}_N*cos(\theta_H) + delta_k_t*cos(\theta_{delta_k}))**2/2)/(Q*\hat{z}_N)],
[                                                                                                                                                            sqrt(2)*Q*\hat{z}_N/2],
[                                                                                                                   -T_t*\hat{z}_N*cos(\theta_H) + delta_k_t*cos(\theta_{delta_k})],
[                                                                                                                   -T_t*\hat{z}_N*sin(\theta_H) + delta_k_t*sin(\theta_{delta_k})]])

In [17]:
sp.Matrix(ki.vec)


Matrix([
[                                                                            sqrt(2)*Q/(2*\hat{x}_N)],
[sqrt(2)*\hat{x}_N*(M_ki**2/2 + k_i_t**2*sin(\theta_{k_i})**2/2 + k_i_t**2*cos(\theta_{k_i})**2/2)/Q],
[                                                                            k_i_t*cos(\theta_{k_i})],
[                                                                            k_i_t*sin(\theta_{k_i})]])

### build R01 Eq.(4.14) https://inspirehep.net/literature/1732230

In [16]:
R01=sp.Abs((ki.get_sqmass())/(Q**2))
R01=R01.simplify()
R01

M_ki**2/Q**2

### build R02 Eq.(4.14) https://inspirehep.net/literature/1732230

In [17]:
R02=sp.Abs((kf.get_sqmass())/(Q**2))
R02=R02.simplify()
R02   

M_kf**2/Q**2

### build R03 Eq.(4.14) https://inspirehep.net/literature/1732230

In [18]:
R03=(dk.t**2)/(Q**2) 
R03=R03.simplify()
R03

delta_k_t**2/Q**2

### build R41 Eq.(100)

In [19]:
R41=sp.Abs((ki*ki)/(k.get_sqmass()))
R41=R41.simplify()
R41

M_ki**2*\hat{z}_N/Abs(-M_kf**2*\hat{z}_N + M_kf**2 - Q**2*\hat{z}_N**2 + Q**2*\hat{z}_N + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2)

### build R42 Eq.(100)

In [20]:
R42=sp.Abs((kf*kf)/(k.get_sqmass()))
R42=R42.simplify()
R42

M_kf**2*\hat{z}_N/Abs(-M_kf**2*\hat{z}_N + M_kf**2 - Q**2*\hat{z}_N**2 + Q**2*\hat{z}_N + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2)

### build R43 Eq.(100)

In [21]:
R43=sp.Abs((dk.t**2)/(k.get_sqmass()))
R43=R43.simplify()
R43

\hat{z}_N*delta_k_t**2/Abs(-M_kf**2*\hat{z}_N + M_kf**2 - Q**2*\hat{z}_N**2 + Q**2*\hat{z}_N + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2)

### build R1

In [22]:
R1=(H*kf)/(H*ki)
R1=R1.simplify()
R1

Q**2*\hat{x}_N*(M_h**2*\hat{z}_N**2 + M_kf**2*z_N**2 + delta_k_t**2*z_N**2)/(\hat{z}_N*(Q**4*z_N**2 + 2*Q**2*T_t*\hat{x}_N*k_i_t*z_N**2*cos(\theta_H - \theta_{k_i}) + \hat{x}_N**2*(M_h**2 + T_t**2*z_N**2)*(M_ki**2 + k_i_t**2)))

### build R2

In [23]:
R2=sp.Abs(k.get_sqmass()/Q**2)
R2=R2.simplify()
R2

Abs(M_kf**2 - \hat{z}_N*(M_kf**2 + Q**2*\hat{z}_N - Q**2 - T_t**2*\hat{z}_N + 2*T_t*delta_k_t*cos(\theta_H - \theta_{delta_k})) + delta_k_t**2)/(Q**2*\hat{z}_N)

### build R3

In [24]:
R3=sp.Abs(kx.get_sqmass()/Q**2)
R3=R3.simplify()
R3

Abs(Q**2*\hat{x}_N*\hat{z}_N*(T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + 2*T_t*\hat{z}_N*k_i_t*cos(\theta_H - \theta_{k_i}) + delta_k_t**2 - 2*delta_k_t*k_i_t*cos(\theta_{delta_k} - \theta_{k_i}) + k_i_t**2) + (-Q**2*(\hat{z}_N - 1) + \hat{x}_N*(M_ki**2 + k_i_t**2))*(Q**2*\hat{x}_N*\hat{z}_N - Q**2*\hat{z}_N + \hat{x}_N*(M_kf**2 + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2)))/(Q**4*\hat{x}_N*\hat{z}_N)

### build R1p

In [25]:
R1p=H*P/Q**2
R1p=R1p.simplify()
R1p

(M**2*x_N**2*(M_h**2 + T_t**2*z_N**2) + Q**4*z_N**2)/(2*Q**4*x_N*z_N)

### build proton rapidity

In [26]:
yp=1/sp.S(2)*sp.log(sp.Abs(P.vec[0]/P.vec[1])).simplify()
yp

log(Q**2/(M**2*x_N**2))/2

### build hadron rapidity

In [27]:
yh=1/sp.S(2)*sp.log(sp.Abs(H.vec[0]/H.vec[1])).simplify()
yh

-log(Q) - log(z_N) + log(M_h**2 + T_t**2*z_N**2)/2

### build incomming parton rapidity

In [28]:
yi=(1/sp.S(2)*sp.log(sp.Abs(ki.vec[0]/ki.vec[1]))).simplify()
yi

log(Q/(\hat{x}_N*sqrt(M_ki**2 + k_i_t**2)))

### build outgoing parton rapidity

In [29]:
yf=(1/sp.S(2)*sp.log(sp.Abs(kf.vec[0]/kf.vec[1]))).simplify()
yf

-log(Q) - log(\hat{z}_N) + log(Abs(M_kf**2 + T_t**2*\hat{z}_N**2 - 2*T_t*\hat{z}_N*delta_k_t*cos(\theta_H - \theta_{delta_k}) + delta_k_t**2))/2

### get Htmax

In [30]:
Wsidis=P+q-H
sp.Matrix(Wsidis.vec)

Matrix([
[-sqrt(2)*Q/2 + sqrt(2)*Q/(2*x_N) - sqrt(2)*(M_h**2/2 + T_t**2*z_N**2*sin(\theta_H)**2/2 + T_t**2*z_N**2*cos(\theta_H)**2/2)/(Q*z_N)],
[                                                                             sqrt(2)*M**2*x_N/(2*Q) - sqrt(2)*Q*z_N/2 + sqrt(2)*Q/2],
[                                                                                                              T_t*z_N*cos(\theta_H)],
[                                                                                                              T_t*z_N*sin(\theta_H)]])

In [31]:
sqHtmax=sp.solve(Wsidis*Wsidis-M**2,T.t**2)[0]*zN**2
sqHtmax

(-M**2*M_h**2*x_N**2 - M**2*Q**2*x_N**2*z_N + M_h**2*Q**2*x_N*z_N - M_h**2*Q**2*x_N + Q**4*x_N*z_N**2 - Q**4*x_N*z_N - Q**4*z_N**2 + Q**4*z_N)/(x_N*(M**2*x_N + Q**2))

In [32]:
sp.factor(sqHtmax.replace(M,0).replace(Mh,0))

Q**2*z_N*(x_N - 1)*(z_N - 1)/x_N

### get rapidity lims

In [33]:
sp.Matrix(H.vec)

Matrix([
[sqrt(2)*(M_h**2/2 + T_t**2*z_N**2*sin(\theta_H)**2/2 + T_t**2*z_N**2*cos(\theta_H)**2/2)/(Q*z_N)],
[                                                                                 sqrt(2)*Q*z_N/2],
[                                                                          -T_t*z_N*cos(\theta_H)],
[                                                                          -T_t*z_N*sin(\theta_H)]])

In [34]:
expr=(H.vec[0]/H.vec[1]).simplify()
rap=(sp.log(expr)/2)#.subs(T.t,H.t/zN)
rap

log((M_h**2 + T_t**2*z_N**2)/(Q**2*z_N**2))/2

## from breit frame to hadron frame via   B->R->B 

### Setup momenta in LC coordenates

In [35]:
H=Vec('H')
q=Vec('q',mod=[1,1,0,0])
P=Vec('P',mod=[1,1,0,0])
T=Vec('T',mod=[0,0,1,1])

q.set_plus(-Q/sp.sqrt(2))
q.set_minus(Q/sp.sqrt(2))
P.set_plus(Q/xN/sp.sqrt(2))
P.set_minus(xN*M**2/sp.sqrt(2)/Q)
H.set_minus(zN*q.vec[1])
H.set_perp((-zN*T).vec[2:])
H.change_plus(Mh)

In [36]:
sp.Matrix(P.vec)

Matrix([
[     sqrt(2)*Q/(2*x_N)],
[sqrt(2)*M**2*x_N/(2*Q)],
[                     0],
[                     0]])

In [37]:
sp.Matrix(q.vec)

Matrix([
[-sqrt(2)*Q/2],
[ sqrt(2)*Q/2],
[           0],
[           0]])

In [38]:
H.vec[0]=H.vec[0].simplify()
sp.Matrix(H.vec)

Matrix([
[sqrt(2)*(M_h**2 + T_t**2*z_N**2)/(2*Q*z_N)],
[                           sqrt(2)*Q*z_N/2],
[                  -T_t*z_N*cos(\theta_{T})],
[                  -T_t*z_N*sin(\theta_{T})]])

### Change momenta to minkowski coordinates

In [39]:
H.LC2MC()
q.LC2MC()
P.LC2MC()

H.simplify_vec()
q.simplify_vec()
P.simplify_vec()

In [40]:
sp.Matrix(P.vec)

Matrix([
[ M**2*x_N/(2*Q) + Q/(2*x_N)],
[                          0],
[                          0],
[-M**2*x_N/(2*Q) + Q/(2*x_N)]])

In [41]:
sp.Matrix(q.vec)

Matrix([
[ 0],
[ 0],
[ 0],
[-Q]])

In [42]:
sp.Matrix(H.vec)

Matrix([
[(M_h**2 + Q**2*z_N**2 + T_t**2*z_N**2)/(2*Q*z_N)],
[                        -T_t*z_N*cos(\theta_{T})],
[                        -T_t*z_N*sin(\theta_{T})],
[(M_h**2 - Q**2*z_N**2 + T_t**2*z_N**2)/(2*Q*z_N)]])

### boost along $P$

In [43]:
bP=P.boost(P)
bH=H.boost(P)
bq=q.boost(P)

bP.simplify_vec()
bH.simplify_vec()
bq.simplify_vec()

In [44]:
sp.Matrix(bP.vec)

Matrix([
[M],
[0],
[0],
[0]])

In [45]:
sp.Matrix(bq.vec)

Matrix([
[-M*x_N/2 + Q**2/(2*M*x_N)],
[                        0],
[                        0],
[-M*x_N/2 - Q**2/(2*M*x_N)]])

In [46]:
sp.Matrix(bH.vec)

Matrix([
[M*M_h**2*x_N/(2*Q**2*z_N) + M*T_t**2*x_N*z_N/(2*Q**2) + Q**2*z_N/(2*M*x_N)],
[                                                  -T_t*z_N*cos(\theta_{T})],
[                                                  -T_t*z_N*sin(\theta_{T})],
[M*M_h**2*x_N/(2*Q**2*z_N) + M*T_t**2*x_N*z_N/(2*Q**2) - Q**2*z_N/(2*M*x_N)]])

### rotation

In [47]:
Z=Vec('Z','MC',[0,0,0,1])
Z.set_z(-1)
cos=bH.get_cos(Z).simplify()
sin=bH.get_sin(Z).simplify()
N=Vec.cross(bH,Z)
sp.Matrix(N.vec)

Matrix([
[                       0],
[ T_t*z_N*sin(\theta_{T})],
[-T_t*z_N*cos(\theta_{T})],
[                       0]])

In [48]:
rbP=bP.rotate(N,cos,sin)
rbH=bH.rotate(N,cos,sin)
rbq=bq.rotate(N,cos,sin)

rbP.simplify_vec()
rbH.simplify_vec()
rbq.simplify_vec()

In [49]:
sp.Matrix(rbP.vec)

Matrix([
[M],
[0],
[0],
[0]])

In [50]:
sp.Matrix(rbq.vec)

Matrix([
[                                                                                                                                                                               -M*x_N/2 + Q**2/(2*M*x_N)],
[                                         Q**2*T_t*z_N**2*(M**2*x_N**2 + Q**2)*cos(\theta_{T})/sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2)],
[                                         Q**2*T_t*z_N**2*(M**2*x_N**2 + Q**2)*sin(\theta_{T})/sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2)],
[(M**2*x_N**2 + Q**2)*(M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)/(2*M*x_N*sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2))]])

In [51]:
sp.Matrix(rbH.vec)

Matrix([
[                                                    M*M_h**2*x_N/(2*Q**2*z_N) + M*T_t**2*x_N*z_N/(2*Q**2) + Q**2*z_N/(2*M*x_N)],
[                                                                                                                             0],
[                                                                                                                             0],
[-sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2)/(2*M*Q**2*x_N*z_N)]])

### boost

In [52]:
cosh,sinh=Vec.get_boost_CS([rbP.get_E(),rbP.get_z()],[P.get_E(),P.get_z()])

In [53]:
Z=Vec('Z','MC',[0,0,0,1])
Z.set_z(1)
sp.Matrix(Z.vec)

Matrix([
[0],
[0],
[0],
[1]])

In [54]:
brbP=rbP.boost(Z,cosh,sinh)
brbq=rbq.boost(Z,cosh,sinh)
brbH=rbH.boost(Z,cosh,sinh)

brbP.simplify_vec()
brbH.simplify_vec()
brbq.simplify_vec()


In [55]:
sp.Matrix(brbP.vec)

Matrix([
[ M**2*x_N/(2*Q) + Q/(2*x_N)],
[                          0],
[                          0],
[-M**2*x_N/(2*Q) + Q/(2*x_N)]])

In [56]:
sp.Matrix(brbq.vec)

Matrix([
[       (M**2*x_N**2 - Q**2)*(M**2*x_N**2 + Q**2)*(-M**2*M_h**2*x_N**2 - M**2*T_t**2*x_N**2*z_N**2 + Q**4*z_N**2 - sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2))/(4*M**2*Q*x_N**2*sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2))],
[                                                                                                                                                                                           Q**2*T_t*z_N**2*(M**2*x_N**2 + Q**2)*cos(\theta_{T})/sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2)],
[                                                                                                                                                                                           Q**2*T_t*z_N**2*(M**2*x_N**2 + Q**2)*sin(\theta_{T})/sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h*

In [57]:
sp.Matrix(brbH.vec)

Matrix([
[ ((M**2*x_N**2 - Q**2)*sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2) + (M**2*x_N**2 + Q**2)*(M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 + Q**4*z_N**2))/(4*M**2*Q**3*x_N**2*z_N)],
[                                                                                                                                                                                                                                                0],
[                                                                                                                                                                                                                                                0],
[(-(M**2*x_N**2 - Q**2)*(M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 + Q**4*z_N**2) - (M**2*x_N**2 + Q**2)*sqrt(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2))/(4*M**2*Q**3*x_N**2*z_N)]])

In [58]:
qT2=brbq.vec[1]**2+brbq.vec[2]**2
qT2=qT2.simplify()
qT2

Q**4*T_t**2*z_N**4*(M**2*x_N**2 + Q**2)**2/(4*M**2*Q**4*T_t**2*x_N**2*z_N**4 + (M**2*M_h**2*x_N**2 + M**2*T_t**2*x_N**2*z_N**2 - Q**4*z_N**2)**2)

In [59]:
def get_massless(exp,replace=[],n=3):
    if len(replace)>0:
        for _ in replace:
            A,B=_
            exp=exp.subs(A,B)
    exp=exp.subs(Q,1/eps)        
    return sp.series(exp,eps,0,n=n).subs(eps,1/Q).removeO()

In [60]:
get_massless(qT2)

2*M**2*T_t**2*x_N**2/Q**2 + T_t**2

# ROWAN ADJUSTMENTS BEGIN

## Replacements

### Original

In [61]:
# ORIGINAL / MC

# theta_H=sp.symbols('theta_H',positive = True, real = True)
# theta_delta_k=sp.symbols('theta_delta_k',positive = True, real = True)
# theta_ki=sp.symbols('theta_ki',positive = True, real = True)

# def set_replacements(exp,option):
#     exp=exp.subs(hzN,zN/zeta)
#     exp=exp.subs(hxN,xN/xi)
#     if option==1: exp=exp.subs(zN,zN2zhA)
#     if option==2: exp=exp.subs(zN,zN2zhB)
#     exp=exp.subs(xN,xN2xb)
#     exp=exp.subs(Mki**2,-Mki**2)
#     exp=exp.subs(-Mki**2 + ki.t**2,abs(-Mki**2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

#     #exp=exp.subs(Mki**2,-Mki2)
#     #exp=exp.subs(-Mki2 + ki.t**2,abs(-Mki2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

#     #exp=exp.subs(Mkf**2,Mkf2)


#     #--need to rewrite angles string to work as python variables
#     exp=exp.subs(sp.symbols(r'\theta_H'),theta_H)
#     exp=exp.subs(r'\theta_{delta_k}',theta_delta_k)
#     exp=exp.subs(r'\theta_{k_i}',theta_ki)

#     return exp

### Rowan Adjusted

In [61]:
# NO SUB

theta_H=sp.symbols('theta_H',positive = True, real = True)
theta_delta_k=sp.symbols('theta_delta_k',positive = True, real = True)
theta_ki=sp.symbols('theta_ki',positive = True, real = True)

def set_replacements(exp,option):
    exp=exp.subs(hzN,zN/zeta)
    exp=exp.subs(hxN,xN/xi)
    if option==1: exp=exp.subs(zN,zN2zhA)
    if option==2: exp=exp.subs(zN,zN2zhB)
    exp=exp.subs(xN,xN2xb)
    # exp=exp.subs(Mki**2,-Mki**2)
    exp=exp.subs(-Mki**2 + ki.t**2,abs(-Mki**2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

    #exp=exp.subs(Mki**2,-Mki2)
    #exp=exp.subs(-Mki2 + ki.t**2,abs(-Mki2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

    #exp=exp.subs(Mkf**2,Mkf2)


    #--need to rewrite angles string to work as python variables
    exp=exp.subs(sp.symbols(r'\theta_H'),theta_H)
    exp=exp.subs(r'\theta_{delta_k}',theta_delta_k)
    exp=exp.subs(r'\theta_{k_i}',theta_ki)

    return exp

def no_replace_set_replacements(exp,option):
    # exp=exp.subs(hzN,zN/zeta)
    # exp=exp.subs(hxN,xN/xi)
    # if option==1: exp=exp.subs(zN,zN2zhA)
    # if option==2: exp=exp.subs(zN,zN2zhB)
    # exp=exp.subs(xN,xN2xb)
    # exp=exp.subs(Mki**2,-Mki**2)
    # exp=exp.subs(-Mki**2 + ki.t**2,abs(-Mki**2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

    #exp=exp.subs(Mki**2,-Mki2)
    #exp=exp.subs(-Mki2 + ki.t**2,abs(-Mki2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

    #exp=exp.subs(Mkf**2,Mkf2)


    #--need to rewrite angles string to work as python variables
    exp=exp.subs(sp.symbols(r'\theta_H'),theta_H)
    exp=exp.subs(r'\theta_{delta_k}',theta_delta_k)
    exp=exp.subs(r'\theta_{k_i}',theta_ki)

    return exp

### list of functions

In [62]:
funcs=[]
funcs.append(['get_xN=%s',xN2xb])
funcs.append(['get_zN1=%s',zN2zhA])
funcs.append(['get_zN2=%s',zN2zhB])
funcs.append(['get_R01=%s',R01])
funcs.append(['get_R02=%s',R02])
funcs.append(['get_R03=%s',R03])
funcs.append(['get_R41=%s',R41])
funcs.append(['get_R42=%s',R42])
funcs.append(['get_R43=%s',R43])
funcs.append(['get_R1=%s',R1])
funcs.append(['get_R2=%s',R2])
funcs.append(['get_R3=%s',R3])
funcs.append(['get_R1p=%s',R1p])
funcs.append(['get_yp=%s',yp])
funcs.append(['get_yh=%s',yh])
funcs.append(['get_yi=%s',yi])
funcs.append(['get_yf=%s',yf])
funcs.append(['get_Htmax2=%s',(sqHtmax)])
funcs.append(['get_rap=%s',rap])
funcs.append(['get_qT2=%s',qT2])


### Original gen_params

In [63]:
# ORIGINAL PARAMS
# def gen_params():
#     params={}
#     params['M']   = 0.938
#     params['M_h'] = 0.139
#     params['x_bj']= 0.1
#     params['z_h'] = 0.01
#     params['Q']   = 2.0
#     params['T_t'] = 0.1
    
#     #params['eta'] = 0
#     params['xi']  = 0.3
#     params['zeta']= 0.01
#     params['delta_k_t']=0.2
#     params['k_i_t']=0.01
#     params['M_ki']=0.1
#     params['M_kf']=0.1
#     params['theta_H']=0.1
#     params['theta_delta_k']=0.2
#     params['theta_ki']=0.3
#     return params

In [64]:
# MC PARAMS
def gen_params():
    params={}
    params['M']   = 0.9383000135421753
    params['M_h'] = 0.1395999938249588
    params['x_bj']= 0.16320896560347234
    params['z_h'] = 0.5194951775358012
    params['Q']   = 1.4789999723434448
    params['T_t'] = 0.7175107733485967
    
    #params['eta'] = 0
    params['xi']  = 0.16835360085766207
    params['zeta']= 0.5105653069816833
    params['delta_k_t']=1.0020180155403615
    params['k_i_t']=0.30531513305968966
    params['M_ki']=0.2903655407963306
    params['M_kf']=0.009899999946001516
    params['theta_H']=-1.6331975808003956
    params['theta_delta_k']=-1.4707750479396895
    params['theta_ki']=-1.0737523225841965
    return params
    

### Original numerical test

In [65]:
# def numerical_test(exp,option):
#     exp=set_replacements(exp,option)
#     args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)

#     func=sp.lambdify(args, exp)
#     keys=['M','M_h','x_bj','z_h','Q','T_t','xi','zeta',\
#           'delta_k_t','k_i_t','M_ki','M_kf','theta_H','theta_delta_k','theta_ki']

#     params=gen_params()
#     args=[params[_] for _ in keys]
#     func_output = func(*args)
#     rat_output = get_R2(*args)
#     print(f"rat_output: {rat_output}\nfunc_output: {func_output}\nHand output: {R2_hand}")
#     return func(*args)

In [66]:
# numerical_test(funcs[10][1],1)

# START HERE Rowan Changes to gen_params, numerical test

#### Importing data from MC

In [67]:
from ratlib import get_R2, get_R1
import uproot as up
MCLund_pref = "/w/hallb-scshelf2102/clas12/users/rojokell/MCLundAnalysis/"

MC = up.concatenate(MCLund_pref + f"OutputFiles/Files_Spring_24/October_13/run_4_100kevents.root:tree_MC")
event = MC[10]
R2_hand = 0

#### original numerical for all subs

In [68]:
# NUMERICAL TEST FOR all SUB
def numerical_test(exp,option):
    rat_params = event
    params = event
    exp=set_replacements(exp,option)
    rat_args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)
    rat_func=sp.lambdify(rat_args, exp)
    # rat_params=gen_params()
    rat_keys=['M','M_h','x','z','Q','qT_from_pT_BF','xi','zeta',\
          'delta_k_t','ki_t','M_ki','M_kf','theta_H','theta_deltak','theta_ki']
    rat_args=[rat_params[_] for _ in rat_keys]
    rat_output = get_R1(*rat_args)
    
    args=(M,Mh,hxN,hzN,Q,T.t,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)
    func=sp.lambdify(args, exp)
    keys=['M','M_h','x_N_hat','z_N_hat','Q','qT_from_pT_BF',\
          'delta_k_t','ki_t','M_ki','M_kf','theta_H','theta_deltak','theta_ki']
    # params=gen_params_no_sub()
    args=[params[_] for _ in keys]
    # func_output = func(*args)
    func_output = rat_func(*rat_args)
    
    print(f"rat_output: {rat_output}\nfunc_output: {func_output}\nHand output: {R2_hand}")
    # return func(*args)

In [69]:
numerical_test(funcs[9][1],1);

rat_output: 0.30231911839616593
func_output: 0.302319118396166
Hand output: 0


#### new numerical test for no subs

In [70]:
# NUMERICAL TEST FOR NO SUB
def no_sub_numerical_test(exp,option):
    rat_params = event
    params = event
    rat_exp=set_replacements(exp,option)
    rat_args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)
    rat_func=sp.lambdify(rat_args, rat_exp)
    # rat_params=gen_params()
    rat_keys=['M','M_h','x','z','Q','qT_from_pT_BF','xi','zeta',\
          'delta_k_t','ki_t','M_ki','M_kf','theta_H','theta_deltak','theta_ki']
    rat_args=[rat_params[_] for _ in rat_keys]
    rat_output = get_R1(*rat_args)

    
    no_sub_exp=no_replace_set_replacements(exp,option)
    args=(M,Mh,hxN,hzN,Q,T.t,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki,xN,zN)
    func=sp.lambdify(args, no_sub_exp)
    keys=['M','M_h','x_N_hat','z_N_hat','Q','qT_from_pT_BF',\
          'delta_k_t','ki_t','M_ki','M_kf','theta_H','theta_deltak','theta_ki','x_N','z_N']
    # params=gen_params_no_sub()
    args=[params[_] for _ in keys]
    # func_output = func(*args)
    func_output = func(*args)
    MC_output = event["R1"]
    print(f"rat_output: {rat_output}\nfunc_output: {func_output}\nMC output: {MC_output}")
    # return func(*args)

In [71]:
no_sub_numerical_test(funcs[9][1],1);

rat_output: 0.30231911839616593
func_output: 0.24736924211380884
MC output: 0.32725387328120376


#### New numerical test for varying subs

In [72]:
def vary_set_replacements(exp,option):
    exp=exp.subs(hzN,zN/zeta)
    exp=exp.subs(hxN,xN/xi)
    if option==1: exp=exp.subs(zN,zN2zhA)
    if option==2: exp=exp.subs(zN,zN2zhB)
    exp=exp.subs(xN,xN2xb)
    exp=exp.subs(Mki**2,-Mki**2)
    # exp=exp.subs(-Mki**2 + ki.t**2,abs(-Mki**2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

    #exp=exp.subs(Mki**2,-Mki2)
    #exp=exp.subs(-Mki2 + ki.t**2,abs(-Mki2 + dk.t**2)) # TODO (AP) I think this is right, but I should confirm

    #exp=exp.subs(Mkf**2,Mkf2)


    #--need to rewrite angles string to work as python variables
    exp=exp.subs(sp.symbols(r'\theta_H'),theta_H)
    exp=exp.subs(r'\theta_{delta_k}',theta_delta_k)
    exp=exp.subs(r'\theta_{k_i}',theta_ki)

    return exp

In [73]:
# Test for varying subs
def vary_sub_numerical_test(exp,option):
    #no need to change rat stuff when varying subs
    rat_params = event
    params = event
    rat_exp=set_replacements(exp,option)
    rat_args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)
    rat_func=sp.lambdify(rat_args, rat_exp)
    # rat_params=gen_params()
    rat_keys=['M','M_h','x','z','Q','qT_from_pT_BF','xi','zeta',\
          'delta_k_t','ki_t','M_ki','M_kf','theta_H','theta_deltak','theta_ki']
    rat_args=[rat_params[_] for _ in rat_keys]
    rat_output = get_R1(*rat_args)

    
    no_sub_exp=vary_set_replacements(exp,option)
    args=(M,Mh,hxN,hzN,Q,T.t,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki,xN,zN,xi,zeta,xb,zh)
    func=sp.lambdify(args, no_sub_exp)
    keys=['M','M_h','x_N_hat','z_N_hat','Q','qT_from_pT_BF',\
          'delta_k_t','ki_t','M_ki','M_kf','theta_H','theta_deltak','theta_ki','x_N','z_N','xi','zeta','x','z']
    # params=gen_params_no_sub()
    args=[params[_] for _ in keys]
    # func_output = func(*args)
    func_output = func(*args)
    MC_output = event["R1"]
    print(f"rat_output: {rat_output}\nfunc_output: {func_output}\nMC output: {MC_output}")
    # return func(*args)

In [74]:
vary_sub_numerical_test(funcs[9][1],1);

rat_output: 0.30231911839616593
func_output: 0.3243497808095164
MC output: 0.32725387328120376


# END ROWAN

In [ ]:
from numpy import sqrt,log,exp,abs
for func in funcs:
    msg ='%15s'%func[0].replace('%s','')
    msg+=' %10f'%numerical_test(func[1],1)
    msg+=' %10f'%numerical_test(func[1],2)

    print(msg)

### generate code

In [65]:
def buildstr(exp,option):
    exp=set_replacements(exp,option)
    args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,)
    func=lambdastr(args, exp)
    func=func.replace('math.','')
    return func

In [66]:
code=['from __future__ import division'] 
code.append('from numpy import sqrt,log,exp,abs')    
for func in funcs:
    code.append(func[0]%buildstr(func[1],1)) # 2 is the option for target region, 1 currect region
code=[l+'\n' for l in code]
F=open('ratlib-current.py','w')
F.writelines(code)
F.close()

In [67]:
code=['from __future__ import division'] 
code.append('from numpy import sqrt,log,exp,abs')    
for func in funcs:
    code.append(func[0]%buildstr(func[1],2)) # 2 is the option for target region, 1 currect region
code=[l+'\n' for l in code]
F=open('ratlib-target.py','w')
F.writelines(code)
F.close()

In [189]:
# MC PARAMS FOR NO SUB
def gen_params_no_sub():
    params={}
    params['M']   = 0.9383000135421753
    params['M_h'] = 0.1395999938249588
    params['x_N_hat']= 0.9592384556245013
    params['z_N_hat'] = 1.0039051085230934
    params['Q']   = 1.4789999723434448
    params['T_t'] = 0.7175107733485967 #qT_from_pT_BF
    
    #params['eta'] = 0
    # params['xi']  = 0.16835360085766207
    # params['zeta']= 0.5105653069816833
    params['delta_k_t']=1.0020180155403615
    params['k_i_t']=0.30531513305968966
    params['M_ki']=0.2903655407963306
    params['M_kf']=0.009899999946001516
    params['theta_H']=-1.6331975808003956
    params['theta_delta_k']=-1.4707750479396895
    params['theta_ki']=-1.0737523225841965
    return params
    

# Here we build the code with jit to make it run faster

In [78]:
def buildstrnew(exp): # 2 is the option for target region, 1 currect region
    exp=set_replacements(exp,1) 
    args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)
    func=lambdastr(args, exp)
    func=func.replace('math.','')
    return func

In [79]:
codenew=['from __future__ import division'] 
codenew.append('from numpy import sqrt,log,exp,abs,cos')    
codenew.append('from numba import jit')    
for func in funcs:
    #codenew.append('@jit')
    string = buildstrnew(func[1]).replace('lambda','@jit\n'+'def '+func[0].replace('=%s','('))
    string = string.replace(':', '): return')
    codenew.append(string)

In [80]:
codenew=[l+'\n' for l in codenew]
F=open('ratlib-current.py','w')
F.writelines(codenew)
F.close()

In [81]:
def buildstrnew(exp):
    exp=set_replacements(exp, 2) # 2 is the option for target region, 1 currect region
    args=(M,Mh,xb,zh,Q,T.t,xi,zeta,dk.t,ki.t,Mki,Mkf,theta_H,theta_delta_k,theta_ki)
    func=lambdastr(args, exp)
    func=func.replace('math.','')
    return func

In [82]:
codenew=['from __future__ import division'] 
codenew.append('from numpy import sqrt,log,exp,abs,cos')    
codenew.append('from numba import jit')    
for func in funcs:
    #codenew.append('@jit')
    string = buildstrnew(func[1]).replace('lambda','@jit\n'+'def '+func[0].replace('=%s','('))
    string = string.replace(':', '): return')
    codenew.append(string)

In [83]:
codenew=[l+'\n' for l in codenew]
F=open('ratlib-target.py','w')
F.writelines(codenew)
F.close()